# KoBERT

### 0. 환경 설정

- **MXNet**: 신경망을 훈련하고 배포하기 위한 오픈 소스 딥러닝 프레임워크 여러 GPU와 기기에서의 효율적인 확장성
- **GluonNLP**: NLP 작업을 위한 도구와 스크립트를 제공하는 MXNet 생태계의 일부
- **tqdm**: 진행 상태 바

In [1]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch>=1.8.1

!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 34.1 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292697 sha256=889b5a95426ef7f80652e8576857cf138b578c1fb771d5c34dfc0014a2f81d26
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-dy08_zlj/kobert-tokenizer_912dd978529c4a54b0b1a7fa5ef018e2
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-dy08_zlj/kobert-tokenizer_912dd978529c4a54b0b1a7fa5ef018e2
  Resolved https://github.com/SKTBrain/KoBERT

### 1. 데이터 준비

##### 1-1. data load

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
path = r'/content/drive/MyDrive/LikeLion/Final Project/Data/감성대화말뭉치(최종데이터)_Training.xlsx'
df_chat = pd.read_excel(path)
df_chat.drop('Unnamed: 0', axis=1, inplace=True)
df_chat.head()

,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


In [ ]:
df_chat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51630 entries, 0 to 51629
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연령      51630 non-null  object
 1   성별      51630 non-null  object
 2   상황키워드   51630 non-null  object
 3   신체질환    51630 non-null  object
 4   감정_대분류  51630 non-null  object
 5   감정_소분류  51630 non-null  object
 6   사람문장1   51630 non-null  object
 7   시스템문장1  51630 non-null  object
 8   사람문장2   51630 non-null  object
 9   시스템문장2  51630 non-null  object
 10  사람문장3   42695 non-null  object
 11  시스템문장3  42695 non-null  object
dtypes: object(12)
memory usage: 4.7+ MB


In [ ]:
# df_chat.head()
emotion_list = df_chat['감정_대분류'].unique().tolist()
for emotion in emotion_list:
    print(f'{emotion}의 소분류 감정:', df_chat[df_chat['감정_대분류'] == f'{emotion}']['감정_소분류'].unique())

분노의 소분류 감정: ['노여워하는' '분노' '성가신' '악의적인' '안달하는' '좌절한' '툴툴대는' '방어적인' '짜증내는' '구역질 나는']
기쁨의 소분류 감정: ['느긋' '만족스러운' '신뢰하는' '신이 난' '안도' '자신하는' '기쁨' '흥분' '편안한' '감사하는']
불안의 소분류 감정: ['걱정스러운' '당혹스러운' '불안' '스트레스 받는' '회의적인' '초조한' '취약한' '두려운' '조심스러운' '혼란스러운']
당황의 소분류 감정: ['당황' '부끄러운' '열등감' '외로운' '고립된' '혐오스러운' '한심한' '남의 시선을 의식하는' '죄책감의' '혼란스러운']
슬픔의 소분류 감정: ['마비된' '비통한' '슬픔' '실망한' '염세적인' '우울한' '후회되는' '눈물이 나는' '낙담한' '환멸을 느끼는']
상처의 소분류 감정: ['배신당한' '버려진' '상처' '억울한' '충격 받은' '고립된' '질투하는' '희생된' '괴로워하는' '가난한, 불우한']


In [ ]:
# 기쁨, 당황, 분노의 소분류 감정 데이터 확인
display(df_chat[df_chat['감정_소분류'] == '안도'])

In [ ]:
display(df_chat[(df_chat['감정_대분류'] == '당황') & (df_chat['감정_소분류'] == '고립된')])
# display(df_chat[(df_chat['감정_대분류'] == '당황') & (df_chat['감정_소분류'] == '혼란스러운')])
# display(df_chat[(df_chat['감정_소분류'] == '남의 시선을 의식하는')])

In [ ]:
# ['노여워하는' '분노' '성가신' '악의적인' '안달하는' '좌절한' '툴툴대는' '방어적인' '짜증내는' '구역질 나는']
display(df_chat[df_chat['감정_소분류'] == '안달하는'])

- 체크해야할 감정 소분류
    - 기쁨 - 여유
    - 당황 - 열등감, 외로운, 고립된, 혐오스러운, 한심한, 남의 시선을 의식하는, 죄책감의, 혼란스러운
    - 불안 - 안달하는

In [ ]:
df_chat['감정_대분류'].value_counts()

불안    9320
분노    9160
상처    9143
슬픔    9125
당황    8756
기쁨    6126
Name: 감정_대분류, dtype: int64

##### 1-2. data preprocessing

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [6]:
# 실험적인 데이터 1
# 정확도: 66%, 60.6%
# data 추출
df_chat_human = df_chat[['감정_대분류', '사람문장1', '사람문장2', '사람문장3']]

# 감정_대분류의 상처는 슬픔, 분노, 불안, 당황이 복합적인 label이므로 사용X
df_chat_human = df_chat_human[df_chat_human['감정_대분류'] != '상처']

# 사람 문장을 토대로 감정이 나뉘어 있기 때문에 사람 문장만 추출해서 사용
# 사람 문장의 경우 마지막 문장은 대부분 감정보다는 해결책, 끝맺음 말이 들어있기 때문에
# 문장이 1개인 경우 문장1 사용
# 문장이 2개인 경우 문장1 사용
# 문장이 3개인 경우 문장1, 문장2 사용
print(df_chat_human.isnull().sum())
df_chat_human_selected = df_chat_human[['감정_대분류', '사람문장1']]
df_chat_human_temp = df_chat_human[df_chat_human['사람문장3'].notna()]
df_chat_human_temp = df_chat_human_temp[['감정_대분류', '사람문장2']]
df_chat_human_temp.columns = ['감정_대분류', '사람문장1']
df_chat_human_selected = pd.concat([df_chat_human_selected, df_chat_human_temp], ignore_index=True, axis=0)
df_chat_human_selected = df_chat_human_selected.rename(columns={'사람문장_통합': '사람문장'})
display(df_chat_human_selected)

# 하이퍼 파라미터 설정
max_len = 64
batch_size = 64
warmup_ratio = 0.1  # if training steps 1000, 100 steps are used for warming up
num_epochs = 3
max_grad_norm = 1  # gradient clipping parameter
                   # if computing the gradient and this norm exceeds `max_grad_norm`,
                   # the gradients are scaled down
log_interval = 200
learning_rate =  5e-5

감정_대분류       0
사람문장1        0
사람문장2        0
사람문장3     7463
dtype: int64


,감정_대분류,사람문장1
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
...,...,...
77506,분노,아내한테 병시중 시킬까 봐 미안하고 걱정돼. 건강을 챙기지 않은 내가 실망스러워.
77507,당황,이젠 아내가 날 돌봐주고 고생해. 건강이 악화되기까지 아무 일도 하지 않은 내 잘못이야.
77508,분노,아무것도 할 수 없는 내가 무가치하게 느껴지고 실망스러워.
77509,불안,마음 같아서는 다 할 수 있는 일인데 이젠 몸이 안 따라와 주니 화만 나.


In [ ]:
# 실험적인 데이터 2
# max_len: 128
# 정확도: 77.1%, 71.5%
# data 추출
df_chat_human = df_chat[['감정_대분류', '사람문장1', '사람문장2', '사람문장3']]

# 감정_대분류의 상처는 슬픔, 분노, 불안, 당황이 복합적인 label이므로 사용X
df_chat_human = df_chat_human[df_chat_human['감정_대분류'] != '상처']

# 사람 문장을 토대로 감정이 나뉘어 있기 때문에 사람 문장만 추출해서 사용
# 사람 문장의 경우 마지막 문장은 대부분 감정보다는 해결책, 끝맺음 말이 들어있기 때문에
# 문장이 1개인 경우 문장1 사용
# 문장이 2개인 경우 문장1 사용
# 문장이 3개인 경우 문장1, 문장2 사용
# 해당 문장들을 통합하여 하나의 사람 문장으로 학습
print(df_chat_human.isnull().sum())
df_chat_human_selected = df_chat_human[['감정_대분류', '사람문장1']]
df_chat_human_temp = df_chat_human[df_chat_human['사람문장3'].notna()]
df_chat_human_temp = df_chat_human_temp[['감정_대분류', '사람문장2']]
df_chat_human_selected = pd.merge(df_chat_human_selected, df_chat_human_temp[['사람문장2']],
                                left_index=True, right_index=True, how='outer')
df_chat_human_selected = df_chat_human_selected.fillna('')
df_chat_human_selected['사람문장_통합'] = df_chat_human_selected['사람문장1'] + df_chat_human_selected['사람문장2']
df_chat_human_selected = df_chat_human_selected[['감정_대분류', '사람문장_통합']]
df_chat_human_selected = df_chat_human_selected.rename(columns={'사람문장_통합': '사람문장'})
display(df_chat_human_selected)

# 하이퍼 파라미터 설정
max_len = 128
batch_size = 64
warmup_ratio = 0.1  # if training steps 1000, 100 steps are used for warming up
num_epochs = 3
max_grad_norm = 1  # gradient clipping parameter
                   # if computing the gradient and this norm exceeds `max_grad_norm`,
                   # the gradients are scaled down
log_interval = 200
learning_rate =  5e-5

감정_대분류       0
사람문장1        0
사람문장2        0
사람문장3     7463
dtype: int64


,감정_대분류,사람문장
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
...,...,...
51622,분노,건강관리를 너무 안 해서 건강이 좋지 않아 졌어. 주변 사람들에게 폐 끼칠까 봐 걱...
51623,당황,이제 몸이 점점 약해진다는 게 느껴져. 아내에게 미안하고 속상한 마음이 들어.이젠 ...
51625,분노,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....
51626,불안,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.마음 ...


In [36]:
# 실험적인 데이터 3
# 정확도: 81.1%, 73.1%
# data 추출
df_chat_human = df_chat[['감정_대분류', '사람문장1', '사람문장2', '사람문장3']]

# 감정_대분류의 상처는 슬픔, 분노, 불안, 당황이 복합적인 label이므로 사용X
df_chat_human = df_chat_human[df_chat_human['감정_대분류'] != '상처']

# 사람 문장들을 모두 통합하여 사용.
print(df_chat_human.isnull().sum())
df_chat_human = df_chat_human.fillna('')
df_chat_human["사람문장_통합"] = df_chat_human['사람문장1'] + df_chat_human['사람문장2'] + df_chat_human['사람문장3']
df_chat_human_selected = df_chat_human[['감정_대분류', '사람문장_통합']]
df_chat_human_selected = df_chat_human_selected.rename(columns={'사람문장_통합': '사람문장'})
display(df_chat_human_selected)

# 하이퍼 파라미터 설정
max_len = 128
batch_size = 64
warmup_ratio = 0.1  # if training steps 1000, 100 steps are used for warming up
num_epochs = 4
max_grad_norm = 1  # gradient clipping parameter
                   # if computing the gradient and this norm exceeds `max_grad_norm`,
                   # the gradients are scaled down
log_interval = 200
learning_rate =  5e-5

감정_대분류       0
사람문장1        0
사람문장2        0
사람문장3     7463
dtype: int64


,감정_대분류,사람문장
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...
...,...,...
51622,분노,건강관리를 너무 안 해서 건강이 좋지 않아 졌어. 주변 사람들에게 폐 끼칠까 봐 걱...
51623,당황,이제 몸이 점점 약해진다는 게 느껴져. 아내에게 미안하고 속상한 마음이 들어.이젠 ...
51625,분노,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....
51626,불안,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.마음 ...


In [ ]:
# 추가적인 데이터 확보

##### 1-3. y label Encoder

In [37]:
# 감정_대분류 label Encoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(df_chat_human_selected['감정_대분류'])

print(set(y_train_encoded))
print(label_encoder.classes_)

df_chat_human_selected['감정'] = y_train_encoded
display(df_chat_human_selected)

{0, 1, 2, 3, 4}
['기쁨' '당황' '분노' '불안' '슬픔']


,감정_대분류,사람문장,감정
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...,2
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,2
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,2
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,2
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...,2
...,...,...,...
51622,분노,건강관리를 너무 안 해서 건강이 좋지 않아 졌어. 주변 사람들에게 폐 끼칠까 봐 걱...,2
51623,당황,이제 몸이 점점 약해진다는 게 느껴져. 아내에게 미안하고 속상한 마음이 들어.이젠 ...,1
51625,분노,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....,2
51626,불안,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.마음 ...,3


In [38]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [39]:
# Torch GPU 설정
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device_type)
device = torch.device(device_type)

cuda


In [40]:
data_list = []
for q, label in zip(df_chat_human_selected['사람문장'], df_chat_human_selected['감정']):
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [41]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 42)

In [42]:
print(len(dataset_train), len(dataset_test))

33989 8498


### 2. 모델 불러오기

In [43]:
# BERTSentenceTransform 수정
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0    0   0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2  # 뒤의 조건이 True가 아니면 AssertError 발생
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [44]:
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup  # dynamically update learning rate

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [45]:
# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()  # 평가 모드 학습 중이 아닐 때 설정
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [46]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [47]:
import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [48]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)  # ?: tok이 아닌 tokenizer가 설정,
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [49]:
first_batch = next(iter(data_train))
print(first_batch)

(array([   2, 4627, 5330, 1100, 7086, 1423, 6004, 5007, 2824, 6116, 2229,
       5761, 3168, 2367, 2195, 5913, 7119,  889, 3168, 1632, 7864, 6855,
         54, 6964, 6025, 6150, 6896, 3351, 4196, 4928, 5017, 5838, 3128,
       4396, 7316, 1370,   54, 3466, 1201, 1423, 6116, 2824, 7848, 7305,
       7088, 5591,   54, 7126, 5788, 4101, 6116, 5010, 5405, 6855,   54,
       1698, 1380, 3803, 2710, 5561, 4045,   54,    3,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1], dtype=int32), array(63, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [50]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

In [51]:
first_batch = next(iter(train_dataloader))
print(first_batch)

[tensor([[   2, 4627, 5330,  ...,    1,    1,    1],
        [   2, 2856, 6144,  ...,    1,    1,    1],
        [   2, 3419, 4627,  ...,    1,    1,    1],
        ...,
        [   2, 3319, 6983,  ...,    1,    1,    1],
        [   2, 1375, 4847,  ...,    1,    1,    1],
        [   2, 1434, 4360,  ...,    1,    1,    1]], dtype=torch.int32), tensor([ 63,  50,  35,  61,  51,  69,  81,  56,  67,  47,  37,  46,  39,  69,
         66,  70,  40,  54,  66,  46,  67,  83,  60,  62,  60,  88,  55,  51,
         41,  70,  72,  56,  42,  46,  26,  44,  54,  60,  42,  65,  61,  45,
         35,  63,  72,  22,  54,  60, 106,  64,  49,  70,  38,  56,  65,  27,
         50,  54,  24,  62,  55,  45,  58,  57], dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([1, 4, 0, 2, 1, 1, 1, 4, 3,

In [52]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5,   # 감정 클래스 수
                 dr_rate = None,
                 params = None):

        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [53]:
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [54]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [55]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)  # 입력 텐서 X의 각 행에 대해 최대값을 찾고 vals와 indices에 저장
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [34]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        """
        Args:
            patience (int): 조기 종료까지의 에포크 수
            min_delta (float): 개선으로 간주되는 최소 변화량
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f'조기 종료: {self.patience} 에포크 동안 개선 없음')
                self.early_stop = True

# Check Point

# # 아래의 코드를 학습 코드에 넣어서 조기 종료 및 모델 상태 저장수행
# # 훈련 루프
# for e in range(num_epochs):
#     # ... [훈련 로직]

#     # 검증 로직 (테스트 데이터셋에서의 손실 계산)
#     model.eval()
#     val_loss = 0.0
#     # ... [검증 로직]

#     # 조기 종료 체크
#     early_stopping(val_loss)
#     if early_stopping.early_stop:
#         print(f"Epoch {e+1}: 조기 종료")
#         break

### 3. 모델 학습

In [57]:
num_epochs = 2

In [58]:
from sklearn.metrics import classification_report

train_history = []
test_history = []
loss_history = []

# 학습
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()  # 훈련 모드로 설정
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):  # dataloader의 진행 상태 시각화
        optimizer.zero_grad()  # 기울기 0으로 초기화
        token_ids = token_ids.long().to(device)  # pytorch의 LongTensor(64비트의 정수형 텐서)로 변환
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()  # 손실을 기반으로 기울기 역전파
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # 기울기 클리핑
        optimizer.step()  # optimizer를 사용하여 가중치 업데이트
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length = valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)

            test_acc += calc_accuracy(out, label)

        print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

        test_history.append(test_acc / (batch_id+1))

  0%|          | 0/532 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.396513432264328 train acc 0.84375
epoch 1 batch id 201 loss 0.3479287624359131 train acc 0.8439832089552238
epoch 1 batch id 401 loss 0.43289661407470703 train acc 0.8495168329177057
epoch 1 train acc 0.8499765037593985


  0%|          | 0/133 [00:00<?, ?it/s]

epoch 1 test acc 0.7318656015037595


  0%|          | 0/532 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.4223179221153259 train acc 0.828125


KeyboardInterrupt: 

In [59]:
# 모델 최종 classification report 출력
all_pred_labels = []
all_true_labels = []

model.eval()
with torch.no_grad():
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        preds = out.detach().cpu().numpy()
        labels = label.cpu().numpy()
        for pred in preds:
            all_pred_labels.append(np.argmax(pred))

        all_true_labels.extend(labels)

print(classification_report(all_true_labels, all_pred_labels))

  0%|          | 0/133 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.94      0.93      0.94      1217
           1       0.66      0.70      0.68      1692
           2       0.67      0.70      0.68      1910
           3       0.74      0.68      0.71      1820
           4       0.73      0.72      0.73      1859

    accuracy                           0.73      8498
   macro avg       0.75      0.75      0.75      8498
weighted avg       0.73      0.73      0.73      8498



In [60]:
print(model)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

- loss 값의 진동 > 작은 batch size?

### 4. 모델 저장

In [ ]:
PATH = '/content/drive/MyDrive/LikeLion/Final Project/Model/'
torch.save(model.state_dict(), PATH + 'model_state_dict_test3_1.pt')

### 5. 모델 Test

In [ ]:
# saved Model load
PATH = '/content/drive/MyDrive/LikeLion/Final Project/Model/model_state_dict.pt'  # state_dict 저장 경로
model = BERTClassifier(bertmodel,  dr_rate = 0.5)
model.load_state_dict(torch.load(PATH))  # CUDA GPU 환경 설정 필요
model.to(device)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
max_len = 128
batch_size = 64

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            probabilities = F.softmax(logits, dim=1)

            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("당황이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("불안이")
            elif np.argmax(logits) == 4:
                test_eval.append("슬픔이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
        print("확률은: ", probabilities)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 이젠 아내가 날 돌봐주고 고생해. 건강이 악화되기까지 아무 일도 하지 않은 내 잘못이야.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


NameError: name 'model' is not defined

##### 참고 레퍼런스
[kobert-demo](https://github.com/ChangZero/koBERT-finetuning-demo/blob/main/kobert_colab.ipynb)